# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [18]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [48]:
np.linalg.norm(BB)


array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
       -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648])

### Do the Jacobi iteration

In [21]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [46]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [96]:
from sklearn.metrics import mean_absolute_error


def Jacobi_iterate(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """
    # 1. Matrix splitting
    D, U, L = np.diag(np.diag(A)), np.triu(A, 1), np.tril(A, -1)

    # 2. Building an inverse matrix
    D_inv = np.linalg.inv(D)

    # 3. Defining objective function
    objective = lambda A, x, b: mean_absolute_error(A @ x, b)
    
    # 4. Initialization of starting x
    x_prev = np.zeros(shape=(A.shape[0]))
    x = np.random.uniform(size=(A.shape[0]))

    # 5. Computing optimization by precomputed components
    x_multiplier = D_inv @ (-L - U)
    offset = D_inv @ b
    
    # 6. Start iteratingter = 0
    n_iter = 0
    while objective(A, x, b) > eps and n_iter < n_max_iters:
        x_prev = x
        x = x_multiplier @ x + offset
        n_iter += 1
        print(f"[{n_iter}]: {objective(A, x, b)}")
        
    return n_iter, x

_, x = Jacobi_iterate(A, b, n_max_iters = 2000)

[1]: 3.5523472974221173
[2]: 1.0583434151820499
[3]: 0.3176659062459442
[4]: 0.09527139039266808
[5]: 0.02857131901060047
[6]: 0.008568765713162348
[7]: 0.002569809967278823
[8]: 0.0007706987060726627
[9]: 0.00023113629018306007
[10]: 6.931890107053155e-05
[11]: 2.0789076142311084e-05
[12]: 6.2347452758258756e-06
[13]: 1.8698304952224216e-06
[14]: 5.607712790907382e-07
[15]: 1.681780398082866e-07
[16]: 5.043741384477829e-08
[17]: 1.5126426232761613e-08
[18]: 4.536488949075102e-09
[19]: 1.3605151095996693e-09
[20]: 4.0802512861481953e-10
[21]: 1.223687273038676e-10
[22]: 3.6698998007178574e-11
[23]: 1.1006194913787404e-11
[24]: 3.3008352995356205e-12
[25]: 9.899265435153738e-13
[26]: 2.96863228443911e-13
[27]: 8.904717241353665e-14
[28]: 2.6701557631625405e-14
[29]: 7.993605777301127e-15
[30]: 2.416122857340497e-15
[31]: 6.845218836204481e-16


In [99]:
# 1. Verficiation of convergence to actual X
np.allclose(A @ x, b)

True

array([0.76711663, 0.70811536, 0.79686718, 0.55776083, 0.96583653,
       0.1471569 , 0.029647  , 0.59389349, 0.1140657 , 0.95080985])

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...